#### Load necessary packages

In [1]:
# Web scraping
from bs4 import BeautifulSoup
import requests

# Data manipulation
import pandas as pd
import numpy as np
import datetime as dt
import os 
import time 

# Database
import psycopg2
import psycopg2.extras
import sqlalchemy
from sqlalchemy import create_engine

In [24]:
links = ['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats', 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
        'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats', 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats', 
        'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats', 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
        'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats', 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats']

team_names = ['Arsenal', 'Chelsea', 'Liverpool', 'Manchester City', 'West Ham', 'Manchester United', 
             'Tottenham', 'Leicester']

os.chdir('../../')
load_dotenv('./src/scraping/config/db_secrets.env')

hostname = os.getenv("HOSTNAME")
database = os.getenv("DATABASE")
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
port = os.getenv("PORT")

team_table_id = 'matchlogs_for'
table_ids = ['stats_standard_11160', 'stats_keeper_adv_11160', 'stats_shooting_11160', 'stats_passing_11160',
            'stats_gca_11160', 'stats_defense_11160', 'stats_possession_11160']

data_types = ['summary', 'keeping', 'shooting', 'passing', 'goal_creation', 'defense', 'possession']

#### Team dataset functions

In [ ]:
def create_team_dataset(table_id, links, team_names):
    
    teams_df = pd.DataFrame()
    
    for link, team_name in zip(links, team_names):
    
        # Create soup object
        fbref_link = requests.get(link).text
        soup = BeautifulSoup(fbref_link, 'lxml')

        # Find the table with team stats
        table = soup.find('table', id = table_id)

        # Get the column names from the first row of the table
        cols = []
        
        # Append team_name to the columns
        cols.append('team_name')

        # Get date column from the row header
        cols.append(table.tbody.tr.th.get('data-stat'))

        # Get other column titles from the specific table cells
        for cell in table.tbody.tr.find_all('td'):
            cols.append(cell.get('data-stat'))

        # Create empty dataframe with column names from created list
        team_data = pd.DataFrame(columns = cols)

        # Iterate over all the rows in the table, and add to empty dataframe
        for row in table.tbody.find_all('tr'):

            # Initialize empty dict for the team
            team_dict = {}
            
            # Get team name from input
            team_dict['team_name'] = team_name

            # Get the match date from row header
            team_dict['date'] = row.th.get('csk')

            # Iterate over all the cells and get the remaining column values
            for cell in row.find_all('td'):
                stat = cell.get('data-stat')
                value = cell.text
                team_dict[stat] = value

             # Append team-match row to dataframe        
            team_data = team_data.append(team_dict, ignore_index=True)
            
            # Drop rows where the match is yet to be played 
            team_data = team_data[team_data['result'] != ""]
            
        teams_df = teams_df.append(team_data, ignore_index=True)
        
    return teams_df

def clean_team_dataset(dataset):
    
    cols_to_keep = ['team_name', 'date', 'comp', 'round', 'dayofweek', 'venue', 'result', 'goals_for', 'goals_against', 'opponent', 
               'xg_for', 'xg_against', 'possession']

    dataset = dataset[cols_to_keep]

    # # Convert date to date format
    dataset['date'] = pd.to_datetime(dataset['date'])
    
    # Keep only premier league games
    dataset = dataset[dataset['comp'].str.strip() == 'Premier League']

    # Strip the round variable to only keep the number
    dataset['round'] = dataset['round'].str.split().str[1]

    # Replace empty space with missing values
    dataset = dataset.replace(r'^\s*$', np.nan, regex=True)
    
    # Drop rows where there is no match data
    dataset = dataset.dropna(subset = ['result'])

    # Create a list for object cols
    object_cols = ['date', 'dayofweek', 'round', 'venue', 'result','opponent']
    
    # Convert round to int64
    dataset['round'] = dataset['round'].astype('int64', errors='ignore')

    # # Except objectcols all columns should be numerical
    for col in dataset.columns:
        if col not in object_cols:
            dataset[col] = dataset[col].astype(float, errors='ignore')
            
    return dataset 

def save_team_dataset(dataset, hostname, database, username, password, port):
    
    engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

    dataset.to_sql('summary_team', engine, if_exists='replace', index = False)

In [ ]:
save_team_dataset(clean_team_dataset(create_team_dataset(team_table_id, links, team_names)), 
                  hostname = hostname, database = database, username = username, password = pwd, port = port_id)

#### Squad dataset functions

In [ ]:
def create_squad_dataset(table_id, links, team_names):
    
    squad_df = pd.DataFrame()
    
    for link, team_name in zip(links, team_names):
    
        fbref_link = requests.get(link).text
        soup = BeautifulSoup(fbref_link, 'lxml')

        # Create a soup object for the all-stats table
        table = soup.find('table', id = table_id)

        # Get the column names from the first row of the table
        cols = []
        
        # Append team_name to the columns
        cols.append('team_name')

        # Get player name column from the row header
        cols.append(table.tbody.tr.th.get('data-stat'))

        # Get other column titles from the specific table cells
        for cell in table.tbody.tr.find_all('td'):
            cols.append(cell.get('data-stat'))

        # Create empty dataframe with column names from created list
        squad_data = pd.DataFrame(columns = cols)

        # Iterate over all the rows in the table, and add to empty dataframe
        for row in table.tbody.find_all('tr'):

            # Initialize empty dict for the player
            squad_dict = {}
            
            # Get team name from input
            squad_dict['team_name'] = team_name

            # Get the player's name from row header
            squad_dict['player'] = row.th.get('csk')

            # Iterate over all the cells and get the remaining column values
            for cell in row.find_all('td'):
                stat = cell.get('data-stat')
                if cell.get('data-stat') != 'matches':
                    value = cell.text
                    squad_dict[stat] = value
                else:
                    link = 'https://fbref.com' + cell.a.get('href')
                    squad_dict[stat] = link

             # Append player row to dataframe        
            squad_data = squad_data.append(squad_dict, ignore_index=True)
            
        squad_df = squad_df.append(squad_data, ignore_index=True)
        
    return squad_df

def clean_squad_dataset(dataset):
    
    # Correct age variable
    dataset['age'] = dataset['age'].str[0:2]

    # Correct nationality variable
    dataset['nationality'] = dataset['nationality'].str.split().str[-1]

    # Replace empty space with missing values
    dataset = dataset.replace(r'^\s*$', np.nan, regex=True)

    # Except first 3 columns, all columns should be numerical
    for col in dataset.columns:
        if col not in ['team_name', 'player', 'nationality', 'position']:
            dataset[col] = dataset[col].astype(float, errors='ignore')
            
    return dataset

def save_squad_dataset(dataset, data_type, hostname, database, username, password, port):
    
    engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

    dataset.to_sql(f'{data_type}_squad', engine, if_exists='replace', index=False)

In [ ]:
for table_id, data_type in zip(table_ids, data_types):

    save_squad_dataset(clean_squad_dataset(create_squad_dataset(table_id, links, team_names)), data_type = data_type, 
                      hostname = hostname, database = database, username = username, password = pwd, port = port_id)

In [22]:
def create_player_dataset(table_id, links, team_names):
    
    all_player_df = pd.DataFrame()
    
    #####################################
    
    for link, team_name in zip(links, team_names):
    
        # Connect to FBref page
        fbref_link = requests.get(link).text
        soup = BeautifulSoup(fbref_link, 'lxml')

        # Create a soup object for the all-stats table
        table = soup.find('table', id = table_id)

        # Create list to get all player links
        player_links = []
        for row in table.tbody.find_all('tr'):
            for cell in row.find_all('td'):
                if cell.get('data-stat') == 'matches':
                    link = 'https://fbref.com' + cell.a.get('href')
            player_links.append(link)
            
        team_player_data = pd.DataFrame()

        # Loop through the links in the player_links list and extract info we need 
        for link in player_links:
            
            individual_player_data = pd.DataFrame()

            # First browse overall stats page 
            player_page = requests.get(link).text
            soup = BeautifulSoup(player_page, 'lxml')

            # Get the div that contains the premier league filter
            player_prem_div = soup.find('div', class_ = 'filter')

            # Loop through each filter and get the link for the filter that corresponds to the premier league
            for filt in player_prem_div.find_all('div', class_ = ''):
                if filt.a.text.strip() == '2021-2022 Premier League':
                    player_prem_link = 'https://fbref.com' + filt.a.get('href')

            # Connect to premier league player stats page
            player_prem_page = requests.get(player_prem_link).text
            soup = BeautifulSoup(player_prem_page, 'lxml')

            # Get player name from the player page
            player_name = soup.find('h1', itemprop="name").span.text

            # Create table object to parse through player stats
            table = soup.find('table', id = 'matchlogs_11160')

            # Loop through each match stats and add to dataframe
            for row in table.tbody.find_all('tr'):

                # Initialize empty dict for the player
                player_dict = {}
                
                # Create a team name column
                player_dict['team_name'] = team_name

                # Get the name from the previously stored variable
                player_dict['name'] = player_name

                # Get the match date from row header
                player_dict['date'] = row.th.get('csk')

                # Iterate over all the cells and get the remaining column values
                for cell in row.find_all('td'):
                    if cell.get('data-stat') != 'match_report':
                        stat = cell.get('data-stat')
                        value = cell.text
                        player_dict[stat] = value

                # Append player row to individual player dataframe        
                individual_player_data = individual_player_data.append(player_dict, ignore_index=True)
                
            # Append individual player data to team-player data
            team_player_data = team_player_data.append(individual_player_data, ignore_index=True)
                
        all_player_df = all_player_df.append(team_player_data, ignore_index=True)
            
    return all_player_df

In [25]:
df = create_player_dataset('stats_keeper_adv_11160', links, team_names)

In [29]:
df[df['team_name'] == 'Liverpool']

,team_name,name,date,dayofweek,round,venue,result,squad,opponent,game_started,...,pct_passes_launched_gk,pens_allowed,pens_att_gk,pens_missed_gk,pens_saved,position,psxg_gk,save_pct,saves,shots_on_target_against
68,Liverpool,Alisson,20210814,Sat,Matchweek 1,Away,W 3–0,Liverpool,Norwich City,Y,...,22.7,0,0,0,0,GK,1.0,100.0,4,4
69,Liverpool,Alisson,20210821,Sat,Matchweek 2,Home,W 2–0,Liverpool,Burnley,Y,...,25.0,0,0,0,0,GK,0.7,100.0,3,3
70,Liverpool,Alisson,20210828,Sat,Matchweek 3,Home,D 1–1,Liverpool,Chelsea,Y,...,13.6,0,0,0,0,GK,0.4,50.0,1,2
71,Liverpool,Alisson,20210912,Sun,Matchweek 4,Away,W 3–0,Liverpool,Leeds United,Y,...,16.7,0,0,0,0,GK,1.5,100.0,4,4
72,Liverpool,Alisson,20210918,Sat,Matchweek 5,Home,W 3–0,Liverpool,Crystal Palace,Y,...,40.9,0,0,0,0,GK,0.2,100.0,2,2
73,Liverpool,Alisson,20210925,Sat,Matchweek 6,Away,D 3–3,Liverpool,Brentford,Y,...,39.1,0,0,0,0,GK,1.4,0.0,0,3
74,Liverpool,Alisson,20211003,Sun,Matchweek 7,Home,D 2–2,Liverpool,Manchester City,Y,...,18.2,0,0,0,0,GK,1.2,33.3,1,3
75,Liverpool,Alisson,None,,,,,,,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,Liverpool,Alisson,20211024,Sun,Matchweek 9,Away,W 5–0,Liverpool,Manchester Utd,Y,...,34.8,0,0,0,0,GK,0.4,100.0,3,3
77,Liverpool,Alisson,20211030,Sat,Matchweek 10,Home,D 2–2,Liverpool,Brighton,Y,...,22.9,0,0,0,0,GK,1.4,66.7,3,6


In [20]:
links

['https://fbref.com/en/players/53af52f3/matchlogs/2021-2022/keeper/Kasper-Schmeichel-Match-Logs']

#### Player dataset functions

In [ ]:
def create_player_dataset(table_id, links, team_names):
    
    player_df = pd.DataFrame()
    
    for link, team_name in zip(links, team_names):
    
        # Connect to FBref page
        fbref_link = requests.get(link).text
        soup = BeautifulSoup(fbref_link, 'lxml')

        # Create a soup object for the all-stats table
        table = soup.find('table', id = table_id)

        # Create list to get all player links
        player_links = []
        for row in table.tbody.find_all('tr'):
            for cell in row.find_all('td'):
                if cell.get('data-stat') == 'matches':
                    link = 'https://fbref.com' + cell.a.get('href')
            player_links.append(link)

        # Get the column names of the eventual dataframe by just going to the first player link page
        player_link = player_links[0]
        player_page = requests.get(player_link).text
        soup = BeautifulSoup(player_page, 'lxml')

        # Get the div that contains the premier league filter
        player_prem_div = soup.find('div', class_ = 'filter')

        for filt in player_prem_div.find_all('div', class_ = ''):
            if filt.a.text.strip() == '2021-2022 Premier League':
                player_prem_link = 'https://fbref.com' + filt.a.get('href')

        # Connect to player prem page
        player_prem_page = requests.get(player_prem_link).text
        soup = BeautifulSoup(player_prem_page, 'lxml')

        # Create table object to parse through
        table = soup.find('table', id = 'matchlogs_11160')

        # Get the column names from the first row of the table
        cols = []
        
        # Append team name 
        cols.append('team_name')

        # Append player name to column name
        cols.append('name')

        # Get match date from the row header
        cols.append(table.tbody.tr.th.get('data-stat'))

        # Get other column titles from the specific table cells
        for row in table.tbody.find_all('tr'):
            if row.get('class') == "unused_sub hidden":
                pass
            else:
                for cell in row.find_all('td'):
                    if cell.get('data-stat') != 'match_report':
                        cols.append(cell.get('data-stat'))

        # Get unique list of columns while preserving order
        variables = list(dict.fromkeys(cols))

        # Create empty dataframe with column names from created list
        player_data = pd.DataFrame(columns = variables)

        # Loop through the links in the player_links list and extract info we need 
        for link in player_links:

            # First browse overall stats page 
            player_page = requests.get(link).text
            soup = BeautifulSoup(player_page, 'lxml')

            # Get the div that contains the premier league filter
            player_prem_div = soup.find('div', class_ = 'filter')

            # Loop through each filter and get the link for the filter that corresponds to the premier league
            for filt in player_prem_div.find_all('div', class_ = ''):
                if filt.a.text.strip() == '2021-2022 Premier League':
                    player_prem_link = 'https://fbref.com' + filt.a.get('href')

            # Connect to premier league player stats page
            player_prem_page = requests.get(player_prem_link).text
            soup = BeautifulSoup(player_prem_page, 'lxml')

            # Get player name from the player page
            player_name = soup.find('h1', itemprop="name").span.text

            # Create table object to parse through player stats
            table = soup.find('table', id = 'matchlogs_11160')

            # Loop through each match stats and add to dataframe
            for row in table.tbody.find_all('tr'):

                # Initialize empty dict for the player
                player_dict = {}
                
                # Create a team name column
                player_dict['team_name'] = team_name

                # Get the name from the previously stored variable
                player_dict['name'] = player_name

                # Get the match date from row header
                player_dict['date'] = row.th.get('csk')

                # Iterate over all the cells and get the remaining column values
                for cell in row.find_all('td'):
                    if cell.get('data-stat') != 'match_report':
                        stat = cell.get('data-stat')
                        value = cell.text
                        player_dict[stat] = value

                # Append player row to dataframe        
                player_data = player_data.append(player_dict, ignore_index=True)
                
            player_df = player_df.append(player_data, ignore_index=True)
            
    return player_df

def clean_player_dataset(dataset):
    
    # Convert date to date format
    dataset['date'] = pd.to_datetime(dataset['date'])

    # Strip the round variable to only keep the number
    dataset['round'] = dataset['round'].str.split().str[1]

    # Replace empty space with missing values
    dataset = dataset.replace(r'^\s*$', np.nan, regex=True)

    # Create a list for object cols
    object_cols = ['team_name', 'name', 'date', 'dayofweek', 'round', 'venue', 'result', 'squad',
                   'opponent', 'game_started', 'position', 'bench_explain']

    # # Except first 3 columns, all columns should be numerical
    for col in dataset.columns:
        if col not in object_cols:
            dataset[col] = dataset[col].astype(float, errors='ignore')
            
    return dataset

def save_player_dataset(dataset, data_type, hostname, database, username, password, port):
    
    engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

    dataset.to_sql(f'{data_type}_player', engine, if_exists='replace', index=False)

In [ ]:
for table_id, data_type in zip(table_ids, data_types):

    save_player_dataset(clean_player_dataset(create_player_dataset(table_id, links, team_names)), data_type = data_type, 
                      hostname = hostname, database = database, username = username, password = pwd, port = port_id)

#### Aggregate scraping function

In [ ]:
def scrape_fbref_data(links, team_names, team_table_id, table_ids, data_types, 
                      hostname, database, username, password, port):
    
    print(f'Scraping team summary')
    save_team_dataset(clean_team_dataset(create_team_dataset(table_id = team_table_id, links = links, team_names = team_names)),
                      hostname = hostname, database = database, username = username, password = pwd, port = port_id)
    
    for table_id, data_type in zip(table_ids, data_types):
        
        print(f'Scraping squad {data_type}')
        save_squad_dataset(clean_squad_dataset(create_squad_dataset(table_id = table_id, links = links, team_names = team_names)), data_type = data_type,
                           hostname = hostname, database = database, username = username, password = pwd, port = port_id)
        
        print(f'Scraping player {data_type}')
        save_player_dataset(clean_player_dataset(create_player_dataset(table_id, links, team_names)), data_type = data_type, 
                            hostname = hostname, database = database, username = username, password = pwd, port = port_id)
        
    print('Scraping complete')

In [ ]:
start = time.time()
scrape_fbref_data(links = links, team_names = team_names, team_table_id = team_table_id, table_ids = table_ids, 
                 data_types = data_types, hostname = hostname, database = database, username = username, 
                  password = pwd, port = port_id)
end = time.time()
print(f'Runtime of scraping function is {end-start}')